In [2]:
from pyomo.environ import *
from __future__ import division # safety with double division
from pyomo.opt import SolverFactory
infinity = float('inf')

In [3]:
model = AbstractModel()
model.name = "Oracle Solver"

In [4]:
model.NumOfWeeks = Param(within=NonNegativeIntegers)
model.Week = RangeSet(1,model.NumOfWeeks)

In [5]:
model.Male = Set()
model.Female = Set()

In [6]:
model.Beams = Param(model.Week, within=NonNegativeIntegers)
model.Assignment = Param(model.Male, model.Female,model.Week, default=0.0, within=Binary)
model.TruthBooth = Param(model.Male, model.Female, default=0.0, within=Binary)

In [7]:
model.PerfectMatch = Var(model.Male, model.Female, within=Binary)

In [8]:
def optimality_condition(model):
    return sum(model.PerfectMatch[i, j] \
               for i in model.Male \
               for j in model.Female)

model.optimization = Objective(rule=optimality_condition, sense=maximize)

In [9]:
def only_one_rule(model, boys):
    return sum(model.PerfectMatch[boys,g] for g in model.Female) <= 1

model.limit_one_rule = Constraint(model.Male, rule=only_one_rule)

In [10]:
def derive_match_rule(model, week):
    return sum((model.PerfectMatch[b,g] * model.Assignment[b,g,week] for b in model.Male for g in model.Female)) == model.Beams[week]
model.match_rule = Constraint(model.Week, rule=derive_match_rule)

In [11]:
def truth_booth_false(model, b, g):
    if (b,g) in model.TruthBooth:
       return model.TruthBooth[b,g]*model.PerfectMatch[b,g] <= 0
    else:
        return Constraint.Skip()
model.failed_truth_booth = Constraint(model.Male, model.Female, rule=truth_booth_false)

In [ ]:
instance = model.create_instance("Wong2_22.dat")

In [ ]:
# Indicate which solver to use
Opt = SolverFactory("appsi_highs")

# Generate a solution
Soln = Opt.solve(instance)
instance.solutions.load_from(Soln)

# Print the output
print("Termination Condition was "+str(Soln.Solver.Termination_condition))
display(instance)
